# Extract tap data from building surface time history in VTK format
The procedure involves the following
1. Collect all the VTK files from their individual OpenFOAM time history folders into one folder with increasing index --- [this workbook]
2. Convert the pressure time history from .VTK to .CSV and save the time history in individual files and one points file --- [ParaView]
3. Map the data to the tap locations --- [this workbook]

In [12]:
import numpy as np
import os
import sys
import warnings
import pandas as pd

import wind
import foam
import windIO


Only edit the following box

In [13]:
collectVTK = True
mapToTap = False

# 'ttuE010_000.0', 'ttuE010_450.2', 'ttuE010_900.1', 'ttuE087_000.0', 'ttuE087_450.2', 'ttuE087_900.0'
caseName = 'ttuE087_450.2'

caseDir = r"D:\tempData_depot\simData_CandC/"+ caseName
vtkDir = r"I:\backup\graham\gra_20_12_13\fmGlbl\final/"+ caseName + r"\postProcessing\wallPressure"
vtkOutDir = caseDir + r"\postProcessing\wallPressure_VTK"
vtkName = 'p_building.vtk'
fldName = 'p'

# tapCoordFile = r'D:\tempData_depot\simData_CandC\tapCoords3D_0deg.csv'
# tapCoordFile = r'D:\tempData_depot\simData_CandC\tapCoords3D_45deg.csv'
tapCoordFile = r'D:\tempData_depot\simData_CandC\tapCoords3D_90deg.csv'


## Collect VTK files into one directory

In [14]:
if collectVTK:
    import shutil
    os.makedirs(vtkOutDir, exist_ok=True)

    vtkTimes = os.listdir(vtkDir)
    vtkTimesNum = np.asarray(vtkTimes,dtype=float)
    vtkTimesSorted = sorted(zip(vtkTimesNum,vtkTimes),key=lambda x: x[0])
    vtkTimesSorted = list(vtkTimesSorted)

    for v in vtkTimesSorted:
        srcFile = vtkDir+'/'+v[1]+'/'+vtkName
        newName = fldName+'_'+str(int(round(v[0]*10000,4))).zfill(7)+'.vtk'
        destFile = vtkOutDir+'/'+newName
        if os.path.isfile(destFile):
            continue
        try:
            shutil.copy(srcFile,destFile)
            print(srcFile+'-->'+newName)
        except:
            print('ERROR! Skipping '+srcFile)

I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0033/p_building.vtk-->p_0000033.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0066/p_building.vtk-->p_0000066.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0099/p_building.vtk-->p_0000099.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0132/p_building.vtk-->p_0000132.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0165/p_building.vtk-->p_0000165.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0198/p_building.vtk-->p_0000198.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0231/p_building.vtk-->p_0000231.vtk
I:\backup\graham\gra_20_12_13\fmGlbl\final/ttuE087_450.2\postProcessing\wallPressure/0.0264/p_building.vtk-->p_0000264.vtk
I:\backup\graham

## Map $p$ to tap locations

In [15]:
if mapToTap:
    tapCoords = pd.read_csv(tapCoordFile)
    tapCoords = tapCoords[['x','y','z']].to_numpy()

    wallPressure_CSV_Dir = caseDir + r"\postProcessing\wallPressure_CSV"

    p,newPts,distance = windIO.extractTapDataFromCSV(wallPressure_CSV_Dir,tapCoords)
    print(f"Shape of p: {p.shape}")

    outFile = caseDir + r"\postProcessing\wallPressure_pOfT_TNGtaps.npy"
    np.save(outFile, p)
